In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import random

sample_size = random.randint(9000, 10000)

data_path = 'cleaned.csv'
test_data_path = 'test_set_VU_DM.csv'

df = pd.read_csv(data_path, nrows=sample_size)
df_test = pd.read_csv(test_data_path, nrows=sample_size)

print(f"Data loaded from {data_path}: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"Data loaded from {test_data_path}: {df_test.shape[0]} rows, {df_test.shape[1]} columns")

print("\nRandom sample from cleaned.csv:")
print(df.sample(n=5, random_state=42))

print("\nSummary statistics of cleaned.csv:")
print(df.describe(include='all'))

print("\nData Types and Memory Usage:")
memory_info = df.info(memory_usage='deep')
print(memory_info)


Data loaded from cleaned.csv: 9078 rows, 36 columns
Data loaded from test_set_VU_DM.csv: 9078 rows, 50 columns

Random sample from cleaned.csv:
      visitor_hist_starrating  visitor_hist_adr_usd  prop_location_score1  \
1400                 0.024487             -0.048958             -0.707107   
8074                -6.027945             -1.434038             -0.602601   
743                  0.024487             -0.048958              0.840890   
1488                 0.024487             -0.048958              0.044031   
267                  0.024487             -0.048958             -0.151918   

      prop_location_score2  prop_starrating  price_usd  gross_bookings_usd  \
1400             -0.334591        -0.171762  -0.009512           -0.033481   
8074             -0.799936        -0.171762  -0.009012           -0.033481   
743               0.812702         1.731145  -0.006366           -0.033481   
1488             -0.556084         0.779692  -0.002230           -0.033481   
267

In [3]:
df.columns

Index(['visitor_hist_starrating', 'visitor_hist_adr_usd',
       'prop_location_score1', 'prop_location_score2', 'prop_starrating',
       'price_usd', 'gross_bookings_usd', 'position', 'comp1_rate',
       'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff', 'click_bool', 'booking_bool', 'srch_id',
       'date_time'],
      dtype='object')

In [4]:
unique_srch_id_test = np.unique(df_test['srch_id'])
unique_srch_id_train = np.unique(df['srch_id'])

missing_srch_id = unique_srch_id_test[~np.isin(unique_srch_id_test, unique_srch_id_train)]
missing_srch_id


array([  3,   7,  10,  13,  14,  16,  18,  19,  23,  24,  27,  32,  33,
        34,  35,  51,  54,  55,  58,  62,  68,  69,  70,  76,  77,  86,
        88,  89,  91,  95,  98, 100, 130, 132, 137, 150, 156, 164, 169,
       171, 173, 182, 186, 197, 200, 206, 209, 212, 215, 227, 228, 232,
       233, 238, 239, 245, 254, 255, 256, 258, 261, 269, 270, 272, 274,
       278, 279, 280, 283, 284, 285, 286, 287, 291, 298, 302, 304, 315,
       318, 319, 323, 327, 334, 341, 347, 349, 352, 358, 363, 364, 368,
       369, 373, 378, 380, 381, 389, 393, 403, 404, 414, 415, 417, 420,
       421, 422, 426, 437, 440, 443, 446, 448, 450, 454, 459, 464, 467,
       481, 482, 485, 487, 488, 492, 493, 494, 495, 496, 499, 504, 505,
       506, 515, 519, 525, 526, 538, 541, 542, 544, 546, 550, 552, 554,
       561, 572, 577])

In [5]:
df.columns

Index(['visitor_hist_starrating', 'visitor_hist_adr_usd',
       'prop_location_score1', 'prop_location_score2', 'prop_starrating',
       'price_usd', 'gross_bookings_usd', 'position', 'comp1_rate',
       'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff', 'click_bool', 'booking_bool', 'srch_id',
       'date_time'],
      dtype='object')

In [6]:
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
import numpy as np

df['target'] = np.select(
    [df['booking_bool'] == 1, (df['booking_bool'] == 0) & (df['click_bool'] == 1)],
    [5, 1],
    default=0
)

scaler = RobustScaler()
rank_scaler = QuantileTransformer(n_quantiles=100, output_distribution='normal')

df['position'] = rank_scaler.fit_transform(df[['position']])
df[['click_bool', 'booking_bool', 'gross_bookings_usd']] = scaler.fit_transform(
    df[['click_bool', 'booking_bool', 'gross_bookings_usd']]
)

df['target'] = (4 * df['booking_bool'] + df['click_bool']).round(2)

print("Data transformation complete. Statistics:")
print(df[['position', 'click_bool', 'booking_bool', 'gross_bookings_usd', 'target']].describe())


Data transformation complete. Statistics:
          position   click_bool  booking_bool  gross_bookings_usd       target
count  9078.000000  9078.000000   9078.000000         9078.000000  9078.000000
mean     -0.125593     0.045495      0.028090            0.040041     0.157854
std       1.371345     0.208398      0.165239            0.707856     0.833553
min      -5.199338     0.000000      0.000000           -1.371486     0.000000
25%      -0.619855     0.000000      0.000000            0.000000     0.000000
50%      -0.037988     0.000000      0.000000            0.000000     0.000000
75%       0.698526     0.000000      0.000000            0.000000     0.000000
max       5.199338     1.000000      1.000000           30.531380     5.000000


In [7]:
def complete_feature_engineering(df):
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['month'] = df['date_time'].dt.month
    df['day_of_week'] = df['date_time'].dt.dayofweek
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)

    count_cols = ['positive_rate_count', 'negative_rate_count', 
                  'positive_inv_count', 'negative_inv_count', 
                  'positive_rate_diff_count', 'negative_rate_diff_count']
    df[count_cols] = 0

    comp_rate_cols = [f'comp{i}_rate' for i in range(1, 9)]
    comp_inv_cols = [f'comp{i}_inv' for i in range(1, 9)]
    comp_rate_percent_diff_cols = [f'comp{i}_rate_percent_diff' for i in range(1, 9)]

    df['avg_rate'] = df[comp_rate_cols].mean(axis=1)
    df['avg_inv'] = df[comp_inv_cols].mean(axis=1)
    df['avg_rate_percent_diff'] = df[comp_rate_percent_diff_cols].mean(axis=1)

    df['positive_rate_count'] = df[comp_rate_cols].gt(0).sum(axis=1)
    df['negative_rate_count'] = df[comp_rate_cols].lt(0).sum(axis=1)

    df['positive_inv_count'] = df[comp_inv_cols].gt(0).sum(axis=1)
    df['negative_inv_count'] = df[comp_inv_cols].lt(0).sum(axis=1)

    df['positive_rate_diff_count'] = df[comp_rate_percent_diff_cols].gt(0).sum(axis=1)
    df['negative_rate_diff_count'] = df[comp_rate_percent_diff_cols].lt(0).sum(axis=1)

    df['location_score'] = df[['prop_location_score1', 'prop_location_score2']].mean(axis=1)
    
    df['star_rating_diff'] = df['visitor_hist_starrating'].sub(df['prop_starrating'])
    df['price_diff'] = df['visitor_hist_adr_usd'].sub(df['price_usd'])
    
    df['price_location_interaction'] = df['price_usd'] * df['location_score']

    df.drop(columns=comp_rate_cols + comp_inv_cols, inplace=True)
    
    return df

df = complete_feature_engineering(df)
df_test = complete_feature_engineering(df_test)


In [8]:
import gc

def normalize_features(input_df, group_key, target_column, take_log10=False):
    epsilon = 1e-4
    if take_log10:
        input_df[target_column] = np.log10(input_df[target_column] + epsilon)
        
    methods = ["mean", "std"]
    df = input_df.groupby(group_key).agg({target_column: methods})
    df.columns = df.columns.droplevel()
    col = {method: target_column + "_" + method for method in methods}
    df.rename(columns=col, inplace=True)
    df_merge = input_df.merge(df.reset_index(), on=group_key)
    df_merge[target_column + "_norm_by_" + group_key] = (
        df_merge[target_column] - df_merge[target_column + "_mean"]
    ) / df_merge[target_column + "_std"]
    df_merge = df_merge.drop(labels=[col["mean"], col["std"]], axis=1)

    gc.collect()
    return df_merge


In [9]:
def aggregate_features(df, group_key="prop_id", target_column="price_usd", 
                       agg_methods=["mean", "median", "min", "max"], 
                       transform_methods={"mean": ["subtract"]}):
    grouped_data = df.groupby(group_key)[target_column].agg(agg_methods).reset_index()

    for method in agg_methods:
        old_name = method
        new_name = f"{method.upper()}_{target_column}_BY_{group_key}"
        grouped_data.rename(columns={old_name: new_name}, inplace=True)

    df = df.merge(grouped_data, on=group_key, how='left')

    for method, functions in transform_methods.items():
        aggregated_col = f"{method.upper()}_{target_column}_BY_{group_key}"
        for function in functions:
            new_col_name = f"{function.upper()}_{target_column}_{method.upper()}"
            if function == "subtract":
                df[new_col_name] = df[target_column] - df[aggregated_col]
            elif function == "divide":
                df[new_col_name] = df[target_column] / df[aggregated_col]

    return df


In [10]:
operations = [
    ('price_usd', 'srch_id', True),
    ('price_usd', 'prop_id', False),
    ('prop_starrating', 'srch_id', False),
]

aggregations = [
    ('prop_id', 'price_usd'),
    ('srch_id', 'prop_starrating'),
    ('srch_id', 'prop_location_score1'),
    ('srch_id', 'prop_location_score2'),
    ('srch_id', 'prop_review_score'),
    ('srch_id', 'promotion_flag'),
    ('srch_destination_id', 'price_usd'),
]

operations2 = [
    ('prop_starrating', 'srch_id', False),
    ('prop_location_score1', 'srch_id', False),
    ('prop_location_score2', 'srch_id', False),
    ('prop_review_score', 'srch_id', False),
]


In [11]:
for target_column, group_key, log in operations:
    df_test = normalize_features(df_test, group_key, target_column, log)

aggregation_methods = ['mean']
transformation_methods = {'mean': ['subtract']}

for group_key, target_column in aggregations:
    df_test = aggregate_features(
        df_test, group_key, target_column, aggregation_methods, transformation_methods
    )

df_test.sort_values(by='srch_id', inplace=True)

for target_column, group_key, log in operations2:
    df_test = normalize_features(df_test, group_key, target_column, log)


In [12]:
columns_to_drop = ['position', 'click_bool', 'booking_bool', 'gross_bookings_usd', 'date_time']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

df_test.drop(columns=['date_time'], inplace=True, errors='ignore')


In [13]:
df_test.columns

Index(['srch_id', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate_percent_diff',
       'comp2_rate_percent_diff', 'comp3_rate_percent_diff',
       'comp4_rate_percent_diff', 'comp5_rate_percent_diff',
       'comp6_rate_percent_diff', 'comp7_rate_percent_diff',
       'comp8_rate_percent_diff', 'month', 'day_of_week', 'is_weekend',
       'positive_rate_count', 'negative_rate_count', 'positive_inv_count',
       'negative_inv_count', 'positive_rate_di

In [14]:
# No HP optimization
# import xgboost as xgb
# from sklearn.model_selection import train_test_split

# # features = ['prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 
# #             'price_usd', 'promotion_flag', 'srch_length_of_stay', 'srch_booking_window', 
# #             'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 
# #             'orig_destination_distance', 'location_score', 'star_rating_diff', 'price_diff',
# #             'price_location_interaction']
# features = [
#     'prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 
#     'price_usd', 'promotion_flag', 'srch_length_of_stay', 'srch_booking_window', 
#     'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 
#     'orig_destination_distance', 'location_score', 'star_rating_diff', 'price_diff',
#     'price_location_interaction', 'avg_rate', 'avg_inv', 'avg_rate_percent_diff',
#     'visitor_hist_starrating', 'visitor_hist_adr_usd', 'srch_query_affinity_score',
#     'month', 'day_of_week', 'is_weekend'
# ]
# X = df[features]
# y = df['target']

# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

# model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
#                 max_depth = 5, alpha = 10, n_estimators = 100)
# model.fit(X_train, y_train)

# df_test['predicted_score'] = model.predict(df_test[features]) 

In [15]:
df = df.replace([np.inf, -np.inf], 0).fillna(0)
df_test = df_test.replace([np.inf, -np.inf], 0).fillna(0)


In [17]:
# import numpy as np
# import pandas as pd
# import xgboost as xgb
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.metrics import mean_squared_error

# features = [
#     'prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 
#     'price_usd', 'promotion_flag', 'srch_length_of_stay', 'srch_booking_window', 
#     'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 
#     'orig_destination_distance', 'location_score', 'star_rating_diff', 'price_diff',
#     'price_location_interaction', 'avg_rate', 'avg_inv', 'avg_rate_percent_diff',
#     'visitor_hist_starrating', 'visitor_hist_adr_usd', 'srch_query_affinity_score',
#     'month', 'day_of_week', 'is_weekend',
#     'positive_rate_count', 'negative_rate_count', 'positive_inv_count', 'negative_inv_count', 
#     'positive_rate_diff_count', 'negative_rate_diff_count', 
#     'price_usd_norm_by_srch_id', 'price_usd_norm_by_prop_id', 
#     'prop_starrating_norm_by_srch_id', 'MEAN_price_usd_BY_prop_id', 
#     'SUBTRACT_price_usd_MEAN', 'MEAN_prop_starrating_BY_srch_id', 
#     'SUBTRACT_prop_starrating_MEAN', 'MEAN_prop_location_score1_BY_srch_id', 
#     'SUBTRACT_prop_location_score1_MEAN', 'MEAN_prop_location_score2_BY_srch_id', 
#     'SUBTRACT_prop_location_score2_MEAN', 'MEAN_prop_review_score_BY_srch_id', 
#     'SUBTRACT_prop_review_score_MEAN', 'MEAN_promotion_flag_BY_srch_id', 
#     'SUBTRACT_promotion_flag_MEAN', 'MEAN_price_usd_BY_srch_destination_id', 
#     'prop_location_score1_norm_by_srch_id', 'prop_location_score2_norm_by_srch_id', 
#     'prop_review_score_norm_by_srch_id'
# ]

# X = df[features]
# y = df['target']

# X_sample, _, y_sample, _ = train_test_split(X, y, test_size=0.9, random_state=69)

# param_grid = {
#     'colsample_bytree': [0.3, 0.6, 0.9],
#     'learning_rate': [0.1, 0.01, 0.001],
#     'max_depth': [3, 5, 7],
#     'n_estimators': [50, 100, 200]
# }

# xgb_reg = xgb.XGBRegressor(objective='reg:squarederror')

# grid_search = GridSearchCV(
#     estimator=xgb_reg, 
#     param_grid=param_grid, 
#     cv=5, 
#     scoring='neg_mean_squared_error', 
#     n_jobs=-1
# )
# grid_search.fit(X_sample, y_sample)

# print(f"Best parameters found: {grid_search.best_params_}")
# print(f"Lowest RMSE found: {np.sqrt(np.abs(grid_search.best_score_))}")

# best_params = grid_search.best_params_
# model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
# model.fit(X, y)

# df_test['predicted_score'] = model.predict(df_test[features])
# y_pred = model.predict(X)
# train_rmse = np.sqrt(mean_squared_error(y, y_pred))
# print(f"Train RMSE: {train_rmse}")

# sorted_recommendations = df_test.sort_values(by=['srch_id', 'predicted_score'], ascending=[True, False])
# sorted_recommendations[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)


In [18]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error


all_features = [
    'prop_starrating', 'prop_review_score', 'prop_location_score1', 'prop_location_score2', 
    'price_usd', 'promotion_flag', 'srch_length_of_stay', 'srch_booking_window', 
    'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 
    'orig_destination_distance', 'location_score', 'star_rating_diff', 'price_diff',
    'price_location_interaction', 'avg_rate', 'avg_inv', 'avg_rate_percent_diff',
    'visitor_hist_starrating', 'visitor_hist_adr_usd', 'srch_query_affinity_score',
    'month', 'day_of_week', 'is_weekend',
    'positive_rate_count', 'negative_rate_count', 'positive_inv_count', 'negative_inv_count', 
    'positive_rate_diff_count', 'negative_rate_diff_count', 
    'price_usd_norm_by_srch_id', 'price_usd_norm_by_prop_id', 
    'prop_starrating_norm_by_srch_id', 'MEAN_price_usd_BY_prop_id', 
    'SUBTRACT_price_usd_MEAN', 'MEAN_prop_starrating_BY_srch_id', 
    'SUBTRACT_prop_starrating_MEAN', 'MEAN_prop_location_score1_BY_srch_id', 
    'SUBTRACT_prop_location_score1_MEAN', 'MEAN_prop_location_score2_BY_srch_id', 
    'SUBTRACT_prop_location_score2_MEAN', 'MEAN_prop_review_score_BY_srch_id', 
    'SUBTRACT_prop_review_score_MEAN', 'MEAN_promotion_flag_BY_srch_id', 
    'SUBTRACT_promotion_flag_MEAN', 'MEAN_price_usd_BY_srch_destination_id', 
    'prop_location_score1_norm_by_srch_id', 'prop_location_score2_norm_by_srch_id', 
    'prop_review_score_norm_by_srch_id'
]


available_features = [feat for feat in all_features if feat in df.columns]


X = df[available_features]
y = df['target']


X_sample, _, y_sample, _ = train_test_split(X, y, test_size=0.9, random_state=69)


param_grid = {
    'colsample_bytree': [0.3, 0.6, 0.9],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200]
}


xgb_reg = xgb.XGBRegressor(objective='reg:squarederror')
grid_search = GridSearchCV(
    estimator=xgb_reg, 
    param_grid=param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    n_jobs=-1
)


grid_search.fit(X_sample, y_sample)


print(f"Best parameters found: {grid_search.best_params_}")
print(f"Lowest RMSE found: {np.sqrt(np.abs(grid_search.best_score_))}")


best_params = grid_search.best_params_
model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X, y)


df_test['predicted_score'] = model.predict(df_test[available_features])
y_pred = model.predict(X)
train_rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"Train RMSE: {train_rmse}")


sorted_recommendations = df_test.sort_values(by=['srch_id', 'predicted_score'], ascending=[True, False])
sorted_recommendations[['srch_id', 'prop_id']].to_csv('submissions.csv', index=False)


In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

data_csr = csr_matrix((df['target'], (df['srch_id'], df['prop_id'])))

svd = TruncatedSVD(n_components=200, random_state=69)
user_factors = svd.fit_transform(data_csr)
prop_factors = svd.components_.T

user_vector_shape = user_factors[0].shape
zero_vector = np.zeros(user_vector_shape, dtype=user_factors.dtype)

user_factors = np.vstack([user_factors, zero_vector, zero_vector])

df_test['predicted_score'] = model.predict(df_test[features])

final_scores = []
for user_id, prop_id, predicted_score in zip(df_test['srch_id'], df_test['prop_id'], df_test['predicted_score']):
    user_vector = user_factors[user_id] if user_id < len(user_factors) else zero_vector
    prop_vector = prop_factors[prop_id] if prop_id < len(prop_factors) else zero_vector
    svd_score = np.dot(user_vector, prop_vector)
    final_score = predicted_score + 0.1 * svd_score
    final_scores.append(final_score)

df_test['final_score'] = final_scores
sorted_recommendations = df_test.sort_values(by=['srch_id', 'final_score'], ascending=[True, False])

sorted_recommendations[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)
